In [0]:
%pip install azure-eventhub

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.0/218.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-f3761616-8bd0-41d8-a9d3-a46a6c4a8ba0
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
storage_account_name = "anilrtlpstg"
access_key = "muGpHzhXHViflMEuSjPpG498HGi9hW11vHFGp/1nyEMlQtsEh5NoOi0Xyr1alXh6o2FjD7ox4nM9+AStGPHsOw=="

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    access_key
)

In [0]:
dbutils.fs.mkdirs("abfss://streaming@anilrtlpstg.dfs.core.windows.net/test")

True

In [0]:
connectionString = "Endpoint=sb://anil-rtlp-ehns.servicebus.windows.net/;SharedAccessKeyName=telemetry-policy;SharedAccessKey=tmGw2UkOTGO7GFgbuN92jpqolDBg+eEla+AEhHe/tZw=;EntityPath=telemetry-hub"

ehConf = {
  'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString),
  'eventhubs.startingPosition': '{"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}'
}

In [0]:
df = spark.readStream \
    .format("eventhubs") \
    .options(**ehConf) \
    .load()

In [0]:
from pyspark.sql.functions import col

json_df = df.selectExpr("CAST(body AS STRING) as json")

In [0]:
from pyspark.sql.types import *

telemetry_schema = StructType([
    StructField("deviceId", StringType()),
    StructField("temperature", DoubleType()),
    StructField("humidity", DoubleType()),
    StructField("timestamp", StringType())
])

In [0]:
from pyspark.sql.functions import from_json

parsed_df = json_df.select(
    from_json(col("json"), telemetry_schema).alias("data")
).select("data.*")

In [0]:
query = parsed_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://streaming@anilrtlpstg.dfs.core.windows.net/checkpoint/final") \
    .start("abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events")

In [0]:
display(dbutils.fs.ls("abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events"))

path,name,size,modificationTime
abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events/_delta_log/,_delta_log/,0,1772235975000
abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events/part-00000-d1184c70-62f8-41e4-8641-e1819da1a440.c000.snappy.parquet,part-00000-d1184c70-62f8-41e4-8641-e1819da1a440.c000.snappy.parquet,3439,1772235984000
abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events/part-00001-f3db4ec9-1daf-4994-ac6e-42e4d0748dc0.c000.snappy.parquet,part-00001-f3db4ec9-1daf-4994-ac6e-42e4d0748dc0.c000.snappy.parquet,3409,1772235984000


In [0]:
spark.read.format("delta").load(
"abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events"
).display()

deviceId,temperature,humidity,timestamp
device-3,29.69,31.22,2026-02-27 22:12:59
device-1,21.65,67.83,2026-02-27 22:13:03
device-4,20.34,33.8,2026-02-27 22:13:07
device-2,30.31,68.26,2026-02-27 22:13:11
device-4,22.49,51.12,2026-02-27 22:13:15
device-1,25.14,72.18,2026-02-27 22:13:19
device-3,34.33,32.02,2026-02-27 22:13:23
device-5,23.04,45.69,2026-02-27 22:13:27
device-2,33.47,75.69,2026-02-27 22:13:31
device-5,21.21,71.32,2026-02-27 22:13:35


In [0]:
# ===============================
# CELL 14 — Read Bronze Delta data
# Bronze = raw streaming data from EventHub
# ===============================

bronze_df = spark.read.format("delta").load(
"abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/events"
)

display(bronze_df)

deviceId,temperature,humidity,timestamp
device-3,29.69,31.22,2026-02-27 22:12:59
device-1,21.65,67.83,2026-02-27 22:13:03
device-4,20.34,33.8,2026-02-27 22:13:07
device-2,30.31,68.26,2026-02-27 22:13:11
device-4,22.49,51.12,2026-02-27 22:13:15
device-1,25.14,72.18,2026-02-27 22:13:19
device-3,34.33,32.02,2026-02-27 22:13:23
device-5,23.04,45.69,2026-02-27 22:13:27
device-2,33.47,75.69,2026-02-27 22:13:31
device-5,21.21,71.32,2026-02-27 22:13:35


In [0]:
# ===============================
# CELL 15 — Silver transformation
# Cleaning + datatype conversion
# ===============================

from pyspark.sql.functions import to_timestamp, col, to_date

silver_df = bronze_df \
    .filter(col("deviceId").isNotNull()) \
    .withColumn("event_time", to_timestamp("timestamp")) \
    .withColumn("event_date", to_date("event_time"))

display(silver_df)

deviceId,temperature,humidity,timestamp,event_time,event_date
device-3,29.69,31.22,2026-02-27 22:12:59,2026-02-27T22:12:59Z,2026-02-27
device-1,21.65,67.83,2026-02-27 22:13:03,2026-02-27T22:13:03Z,2026-02-27
device-4,20.34,33.8,2026-02-27 22:13:07,2026-02-27T22:13:07Z,2026-02-27
device-2,30.31,68.26,2026-02-27 22:13:11,2026-02-27T22:13:11Z,2026-02-27
device-4,22.49,51.12,2026-02-27 22:13:15,2026-02-27T22:13:15Z,2026-02-27
device-1,25.14,72.18,2026-02-27 22:13:19,2026-02-27T22:13:19Z,2026-02-27
device-3,34.33,32.02,2026-02-27 22:13:23,2026-02-27T22:13:23Z,2026-02-27
device-5,23.04,45.69,2026-02-27 22:13:27,2026-02-27T22:13:27Z,2026-02-27
device-2,33.47,75.69,2026-02-27 22:13:31,2026-02-27T22:13:31Z,2026-02-27
device-5,21.21,71.32,2026-02-27 22:13:35,2026-02-27T22:13:35Z,2026-02-27


In [0]:
# ===============================
# CELL 16 — Write Silver Delta
# ===============================

silver_df.write.format("delta") \
.mode("overwrite") \
.save("abfss://streaming@anilrtlpstg.dfs.core.windows.net/silver/events")

In [0]:
# ===============================
# CELL 17 — Validate Silver table
# ===============================

display(
spark.read.format("delta").load(
"abfss://streaming@anilrtlpstg.dfs.core.windows.net/silver/events"
))

deviceId,temperature,humidity,timestamp,event_time,event_date
device-3,29.69,31.22,2026-02-27 22:12:59,2026-02-27T22:12:59Z,2026-02-27
device-1,21.65,67.83,2026-02-27 22:13:03,2026-02-27T22:13:03Z,2026-02-27
device-4,20.34,33.8,2026-02-27 22:13:07,2026-02-27T22:13:07Z,2026-02-27
device-2,30.31,68.26,2026-02-27 22:13:11,2026-02-27T22:13:11Z,2026-02-27
device-4,22.49,51.12,2026-02-27 22:13:15,2026-02-27T22:13:15Z,2026-02-27
device-1,25.14,72.18,2026-02-27 22:13:19,2026-02-27T22:13:19Z,2026-02-27
device-3,34.33,32.02,2026-02-27 22:13:23,2026-02-27T22:13:23Z,2026-02-27
device-5,23.04,45.69,2026-02-27 22:13:27,2026-02-27T22:13:27Z,2026-02-27
device-2,33.47,75.69,2026-02-27 22:13:31,2026-02-27T22:13:31Z,2026-02-27
device-5,21.21,71.32,2026-02-27 22:13:35,2026-02-27T22:13:35Z,2026-02-27


In [0]:
# ============================================
# GOLD LAYER — Streaming aggregation
# Business-level metrics per device
#
# Metrics:
#  - Average temperature per device
#  - Average humidity per device
#  - Total number of events per device
# ============================================

from pyspark.sql.functions import avg, count

gold_df = parsed_df.groupBy("deviceId").agg(
    avg("temperature").alias("avg_temperature"),
    avg("humidity").alias("avg_humidity"),
    count("*").alias("event_count")
)

In [0]:
# ============================================
# GOLD LAYER — Write aggregated results to Delta
#
# Streaming aggregation uses COMPLETE mode because
# groupBy results must be rewritten every trigger
#
# Checkpoint ensures fault tolerance and recovery
# ============================================

gold_query = gold_df.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", "abfss://streaming@anilrtlpstg.dfs.core.windows.net/checkpoint/gold") \
    .start("abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/gold")

In [0]:
# ============================================
# GOLD LAYER — Validate aggregated table
# Reading Delta table to confirm metrics
# ============================================

spark.read.format("delta").load(
"abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/gold"
).display()

deviceId,avg_temperature,avg_humidity,event_count
device-3,28.18490909090909,56.24727272727272,55
device-1,27.176153846153852,57.32403846153845,52
device-2,28.928285714285717,53.15542857142857,35
device-5,26.69089285714286,56.56339285714286,56
device-4,26.876346153846153,53.54076923076923,52


In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold_metrics
USING DELTA
AS SELECT *
FROM delta.`abfss://streaming@anilrtlpstg.dfs.core.windows.net/delta/gold`

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold_metrics

deviceId,avg_temperature,avg_humidity,event_count
device-1,27.176153846153852,57.32403846153845,52
device-2,28.928285714285717,53.15542857142857,35
device-5,26.69089285714286,56.56339285714286,56
device-4,26.876346153846153,53.54076923076923,52
device-3,28.18490909090909,56.24727272727272,55


In [0]:
for s in spark.streams.active:
    s.stop()